#Processing of the MQ sensors' logs

In this **project**, we'll build a neural network and use it to predict food degradation over time.

In [ ]:
#Importing modules
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

## Load and prepare the data

A critical step in working with neural networks is preparing the data correctly.

In [ ]:
data_path = 'mq_sensors_logs_feta_2_kai_telos.csv'

df_feta = pd.read_csv(data_path)
# Watch all columns
pd.options.display.max_columns = None

# Add value 0 for first day and 1 for rest.
df_feta.loc[0:3600, 'day'] = 0
df_feta.loc[3600:, 'day'] = 1

#rides_feta.head(10)
print(df_feta[3595:3605])

##Checking out the data
This dataset has logs from various different simple air sensors that are collected during food degradation over a period of time. Logs are collected in intervals of 1 minute so for one day we have 1440 entries. Check the basic set up for the [MQ-2 sensor](https://medium.com/p/5c7e2338267f) and the whole [bunch of air sensors](https://medium.com/p/b4523540f63d).

In [ ]:
df_feta[2900:].plot(x='Timestamp', y='Raw_value_MQ2')

##Normalizing the values
We are dividing every log by 65472 wich is the maximum value each sensor can give. For more info read [this](https://github.com/adafruit/Adafruit_CircuitPython_MCP3xxx/blob/main/adafruit_mcp3xxx/analog_in.py#L50-L54).

In [ ]:
sensor_names = ['Raw_value_MQ2', 'Raw_value_MQ3', 'Raw_value_MQ4', 'Raw_value_MQ135', 'Raw_value_MQ6', 'Raw_value_MQ7', 'Raw_value_MQ8', 'Raw_value_MQ9']
sensor_names_with_day = ['Raw_value_MQ2', 'Raw_value_MQ3', 'Raw_value_MQ4', 'Raw_value_MQ135', 'Raw_value_MQ6', 'Raw_value_MQ7', 'Raw_value_MQ8', 'Raw_value_MQ9', 'day']

MAX_VALUE = 65472
for each in sensor_names:
    df_feta.loc[:, each] = df_feta[each] / MAX_VALUE

df_feta[3595:3605]

In [ ]:
#Plot the diagram with normalized data
df_feta[2900:].plot(x='Timestamp', y='Raw_value_MQ2')
df_feta[2900:].plot(x='Timestamp', y='Raw_value_MQ3')
df_feta[2900:].plot(x='Timestamp', y='Raw_value_MQ4')
df_feta[2900:].plot(x='Timestamp', y='Raw_value_MQ135')

In [ ]:
fields = df_feta[sensor_names_with_day]

In [ ]:
#Print a dataframe with normalized values and the day
fields[3595:3605]

### Splitting the data into training, testing, and validation sets

We'll save the data for the first 50 minutes of each day as test set and split the rest as training and validation sets.

In [ ]:
# Test data for 100 minutes
test_data_first_day = fields[2900:2950]
test_data_second_day = fields[3600:3650]
test_data_two_days = [test_data_first_day, test_data_second_day]
# we DO NOT shuffle the test set
test_data = pd.concat(test_data_two_days)


# Now remove the test data from the data set (USE VALUES UNTIL 3000 ENTRIES)
data_first_day = fields[2950:3600]
data_second_day = fields[3650:]
data_two_days = [data_first_day, data_second_day]
data_rest = pd.concat(data_two_days)
# SHUFFLE the dataset before splitting into training and validation and reset the index
data_rest = data_rest.sample(frac=1).reset_index(drop=True)
print(data_rest[:10])

# Separate the data into features and targets
target_field = ['day']
data_features, data_targets = data_rest.drop(target_field, axis=1), data_rest[target_field]
test_features, test_targets = test_data.drop(target_field, axis=1), test_data[target_field]


In [ ]:
# Create test features list.
test_features_list=[]

for row in test_features.iterrows():
    index, data = row
    test_features_list.append(data.tolist())

# Create test targets list.
test_targets_list=[]

for row in test_targets.iterrows():
    index, data = row
    test_targets_list.append(data.tolist())


# Create data features list.
data_features_list=[]

for row in data_features.iterrows():
    index, data = row
    data_features_list.append(data.tolist())

# Create data targets list.
data_targets_list=[]

for row in data_targets.iterrows():
    index, data = row
    data_targets_list.append(data.tolist())

In [ ]:
'''
print(data_features_list[:10])
print(data_targets_list[:10])
print(len(data_targets_list))
'''

In [ ]:
# Split into training and validation data
# Hold out the last 500 entries or so of the remaining data as a validation set
train_features, train_targets = np.array(data_features_list[:700]), np.array(data_targets_list[:700])
val_features, val_targets = np.array(data_features_list[700:]), np.array(data_targets_list[700:])
test_features, test_targets = np.array(test_features_list[:]), np.array(test_targets_list[:])

In [ ]:
'''
print(len(train_features))
print(len(train_targets))


print(len(val_features))
print(len(val_targets))
'''

In [ ]:
# Convert targets to categorical
import tensorflow as tf

train_targets_categorical = tf.keras.utils.to_categorical(train_targets, dtype ="uint8")
val_targets_categorical = tf.keras.utils.to_categorical(val_targets, dtype ="uint8")
test_targets_categorical = tf.keras.utils.to_categorical(test_targets, dtype ="uint8")

In [ ]:
print(test_targets_categorical[48:53])

##Build a Keras model to train the dataset.

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model=Sequential()
model.add(Dense(256, input_dim=8,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2))

print(model.summary())

In [ ]:
from tensorflow.keras.optimizers import Adam


adam = Adam(learning_rate=0.000001)
# https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function
model.compile(loss='categorical_crossentropy', 
             optimizer=adam, 
             metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint  

batch_size = 50
num_epochs = 500

checkpoint = ModelCheckpoint(filepath='food_quality.weights.best.h5', verbose=1, 
                               save_best_only=True)

callbacks_list = [checkpoint]

history = model.fit(train_features, train_targets_categorical, validation_data=(val_features, val_targets_categorical), batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list)

In [ ]:
#Get history
history_dict = history.history
history_dict.keys()

#dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "g" is for "solid green line"
plt.plot(epochs, loss, 'g', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#Evaluate model with the test set
print(len(test_features))
scores = model.evaluate(test_features, test_targets_categorical, verbose=1)

In [ ]:
#Predict an output with one entry
test_features = np.array(test_features_list[52])
test_features = np.expand_dims(test_features, axis=0)
print(test_features)
score = model.predict(test_features, verbose=1)
print(score)
##Show the argmax label
print(tf.math.argmax(score[0]))

##Reconstruct a new model from the .h5 format

In [ ]:
reconstructed_model = keras.models.load_model("/content/food_quality.weights.best.h5")

In [ ]:
#Evaluate the reconstructed model with an entry from the test set
score = reconstructed_model.predict(test_features, verbose=1)
print(score)
print(tf.math.argmax(score[0]))

In [ ]:
#Save the model to the saved model format
reconstructed_model.save('/content/saved_model')

##Convert to tflite

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('/content/saved_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('food_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/food_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

interpreter.set_tensor(input_details[0]['index'], tf.cast(test_features, tf.float32))

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)